In [1]:
import re
import numpy as np
import csv
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Import Dataset
def get_text(file):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        corpus_lst = []
        for row in csv_reader:
            corpus_lst.append(row[18])
            line_count += 1
            print("row", line_count)
    return corpus_lst

file = "airbnb.csv"
data = get_text(file)

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))


# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])


# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

nlp = spacy.load('en_core_web_sm')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])


# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])


# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis




row 1
row 2
row 3
row 4
row 5
row 6
row 7
row 8
row 9
row 10
row 11
row 12
row 13
row 14
row 15
row 16
row 17
row 18
row 19
row 20
row 21
row 22
row 23
row 24
row 25
row 26
row 27
row 28
row 29
row 30
row 31
row 32
row 33
row 34
row 35
row 36
row 37
row 38
row 39
row 40
row 41
row 42
row 43
row 44
row 45
row 46
row 47
row 48
row 49
row 50
row 51
row 52
row 53
row 54
row 55
row 56
row 57
row 58
row 59
row 60
row 61
row 62
row 63
row 64
row 65
row 66
row 67
row 68
row 69
row 70
row 71
row 72
row 73
row 74
row 75
row 76
row 77
row 78
row 79
row 80
row 81
row 82
row 83
row 84
row 85
row 86
row 87
row 88
row 89
row 90
row 91
row 92
row 93
row 94
row 95
row 96
row 97
row 98
row 99
row 100
row 101
row 102
row 103
row 104
row 105
row 106
row 107
row 108
row 109
row 110
row 111
['description']
[['description']]
[[(0, 1)]]
[(0,
  '0.019*"music" + 0.010*"condo" + 0.008*"radio" + 0.008*"record" + '
  '0.008*"wireless" + 0.007*"row" + 0.006*"sleeper" + 0.004*"serta" + '
  '0.004*"publishing" + 0.00

C:\Users\Amber\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.235539  0.028280       1        1  25.153940
2     -0.138357 -0.139587       2        1  19.487495
18    -0.188117 -0.131781       3        1   7.386392
7     -0.143783  0.166583       4        1   7.373932
9     -0.193724 -0.120000       5        1   6.642939
4     -0.010506 -0.050607       6        1   5.569452
5     -0.060948  0.118519       7        1   5.211039
13    -0.075939  0.197803       8        1   5.043881
19    -0.104441  0.124701       9        1   3.662238
12    -0.027133 -0.144845      10        1   3.479636
15     0.098173 -0.000015      11        1   2.901535
8      0.127520 -0.007920      12        1   1.856863
6      0.134806 -0.006387      13        1   1.653435
11     0.121005 -0.005293      14        1   1.061840
3      0.117816 -0.009131      15        1   0.913467
1      0.119831 -0.005643      16        1   0.712257
10     0.118791 -0.003384      17        1   0.680549
0      0.116891 -0.002345      18        1   0.506116
16     0.117803 -0.003541      19        1   0.485901
14     0.105852 -0.005408      20        1   0.217094, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
90    Default  130.000000       bedroom  130.000000  30.0000  30.0000
145   Default  118.000000     apartment  118.000000  29.0000  29.0000
56    Default  117.000000          room  117.000000  28.0000  28.0000
425   Default   64.000000          free   64.000000  27.0000  27.0000
400   Default   26.000000     manhattan   26.000000  26.0000  26.0000
10    Default   92.000000           bed   92.000000  25.0000  25.0000
29    Default   70.000000          home   70.000000  24.0000  24.0000
92    Default   61.000000         block   61.000000  23.0000  23.0000
45    Default   72.000000  neighborhood   72.000000  22.0000  22.0000
127   Default   59.000000         queen   59.000000  21.0000  21.0000
335   Default   59.000000          area   59.000000  20.0000  20.0000
18    Default   48.000000      downtown   48.000000  19.0000  19.0000
441   Default   17.000000        studio   17.000000  18.0000  18.0000
89    Default   30.000000         beach   30.000000  17.0000  17.0000
126   Default   38.000000       private   38.000000  16.0000  16.0000
87    Default   51.000000      bathroom   51.000000  15.0000  15.0000
538   Default   19.000000         check   19.000000  14.0000  14.0000
30    Default   43.000000         house   43.000000  13.0000  13.0000
80    Default   62.000000    restaurant   62.000000  12.0000  12.0000
417   Default   13.000000        avenue   13.000000  11.0000  11.0000
7     Default   36.000000     available   36.000000  10.0000  10.0000
139   Default   40.000000            tv   40.000000   9.0000   9.0000
807   Default   20.000000       cottage   20.000000   8.0000   8.0000
35    Default   72.000000        locate   72.000000   7.0000   7.0000
49    Default   36.000000       parking   36.000000   6.0000   6.0000
48    Default   49.000000          park   49.000000   5.0000   5.0000
27    Default   49.000000         guest   49.000000   4.0000   4.0000
358   Default   59.000000         great   59.000000   3.0000   3.0000
55    Default   31.000000         right   31.000000   2.0000   2.0000
178   Default   44.000000        street   44.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
1992  Topic20    0.008840     sculpture    0.714436   1.7404  -7.7085
1993  Topic20    0.008840      security    0.714436   1.7404  -7.7085
1982  Topic20    0.008840        kindly    0.714436   1.7404  -7.7085
1990  Topic20    0.008840     respected    0.714436   1.7404  -7.7085
1988  Topic20    0.008840  registration    0.714436   1.7404  -7.7085
1985  Topic20    0.008840        permit    0.714436   1.7404  -7.7085
1986  Topic20    0.008840      pictured

In [2]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = 'path/to/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Amber\\AppData\\Local\\Temp\\e88102_state.mallet.gz'

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()



In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))
    
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))



In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)